### Comparing Planet Compositions

This notebook imports `cumulative.csv`, a file produced from a grid whose sampling was *uniformly distributed* across a 1-sigma range of refractory element ratios derived from stellar abundances reported in the [Hypatia Catalog Database](https://www.hypatiacatalog.com/hypatia) described in [Hinkel et al. (2014, AJ, 148, 54)](https://doi.org/10.1088/0004-6256/148/3/54). Development of the catalog was supported by NASA's Nexus for Exoplanet System Science (NExSS) research coordination network and the Vanderbilt Initiative in Data-Intensive Astrophysics (VIDA).

#### A few quick notes about cumulative.csv, and the model parameters embedded in it:
* no Fe allowed in the mantle - only Mg, Ca, Si, Al, and O
* stipulated 33% core by mass, core only Fe (so no light elements)
* mass percents shown are percents of the *total* planet mass - get.adds_up() normalizes so that only the mantle is considered
* 1700K (?) mantle potential temperature assumed - changing T changes mineralogy slightly, but not drastically
* model details + a recent implementation: [Unterborn & Panero (2019, JGR: Planets, 124 7)](doi.org/10.1029/2018JE005844)

### Import external packages and internal modules

In [ ]:
#Import external packages
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.stats as stats
import pandas as pd
%matplotlib inline

#Import internal packages
import evolve
import plot
import fromexo
import getall as get
from mineralDB import minerals
from constants import *

# Convenience functions
Pe = lambda n: format(n, '.4e')
Pf = lambda n: format(n, '.4f')

### User-input values, used across all model runs

In [ ]:
method = 'static'   # static or dynamic calculation of thermal parameters
Pref = 5.0          # Reference pressure for Cp and alpha, GPa
Ts = 300.0          # surface temperature, K
Tp0 = 2000.          # starting mantle potential temperature in K        Earth = 2000.0 (initial), 1600 (present)
Qpl = 1.0           # Relative heat production per kg mantle, vs Earth  Earth = 1.0
tmax = 4.55         # ending time, in Ga                                Earth = 4.55
output_file='compare_compositions_results.csv'
output_file_2='bulk_elemental_fraction.csv'

In [ ]:
files = fromexo.planets_from_summary()
print(len(files.keys()), 'planets to analyze, which may contain:')
posskeys=[]
for f in files:
    for i in files[f]:
        if i not in posskeys:
            posskeys.append(i)
print(', '.join(sorted(posskeys)))

### List what aspects of a planet you'd like shown, w.r.t. thermal evolution pathways.

In [ ]:
# Currently, column values in plottable array are:
#            [t, Tp, Ra, production, loss, production / loss, 
#             Mg, Si, Ca, Al, MgSi, alpha, cp, Water]

columnkeys = ['time', 'temp', 'rayleigh', 'production', 'loss', 'urey', 
             'alpha', 'cp', 'Water', 'Mg', 'Si', 'Ca', 'Al', 'MgSi']

ofinterest0 = 'MgSi'
ofinterest1 = 'Water'
ofinterest2 = 'cp'

out = open(output_file, 'w+')
out.write('file,Mg/Si,Ca/Si,Al/Si,alpha,Cp,k,temp(K),Rayleigh,HeatLoss(W),UreyRatio,Water\n')
out2 = open(output_file_2, 'w+')
out2.write('f_Mg,f_Si,f_Ca,f_Al,alpha,cp,k,tempK,Water\n')
print()

### Calculate thermal evolution pathways

In [ ]:
Hts = []  # A list of lists; column names are in columnkeys
sep = ','  # Outputs will be comma-separated value files (*.csv)
nplanets = 0
print('Progress so far:')
for file in files:

    Mpl = files[file]['Mass_Me']
    Rpl = files[file]['Radius_Re']
    planet = {'Mpl': Mpl, 'Rpl': Rpl, 'Qpl': Qpl, 'Tp0': Tp0}
    planet['Mp'] = files[file]['Mass_kg']
    planet['Mc'] = planet['Mp'] * files[file]['CMF']
    planet['Rp'] = files[file]['Radius_m']
    planet['Rc'] = planet['Rp'] * files[file]['CRF']
    planet['d'] = files[file]['Mantle_depth']
    planet['Vm'] = files[file]['Mantle_vol']
    planet['Sa'] = 4 * np.pi * planet['Rp']**2
    planet['pm'] = files[file]['Mantle_rho']
    planet['g'] = get.Grav * planet['Mp']/(planet['Rp']**2)
    planet['Pcmb'] = files[file]['CMBP']
    planet['Tcmb'] = get.CMB_T(planet['Rp'], planet['Tp0'])
    planet['Pref'] = Pref
    planet['Qp'] = DEFAULT['Qp']
    planet['Ts']=300.0
    planet['beta']=0.3


    # Get elemental abundances for easy plotting
    MgSi = 1 / files[file]['SiMg']
    CaSi = files[file]['CaMg'] * MgSi
    AlSi = files[file]['AlMg'] * MgSi
    totmol = MgSi + CaSi + AlSi + 1
    Mg = MgSi / totmol
    Ca = CaSi / totmol
    Al = AlSi / totmol
    Si = 1 / totmol

    # Given what your planet's made of, find out its water storage capacity and rheological constraints.
    composition = files[file]['composition']
    composition = get.adds_up(composition)
    planet['c1'], planet['Ev'], planet['visc0'] = DEFAULT['c1'], DEFAULT['Ev'], DEFAULT['visc0']
    thermals = get.thermals_at_P_ave(composition, planet['Pref'])
    Water = get.average_property(composition, 'water', 0.0)

    # Now start the evolutionary clock...
    # Be sure to keep Hts, Tp, and t=0.0 here, so at the end of each run, values are reset
    # and the clock starts again.
    Tp = Tp0
    dt = 0.01
    t = 0.0
    
    if method == 'static':
        #alpha, cp, k = get.Tdep_thermals(thermals, 1625)
        planet['Cp'] = files[file]['Cp']
        planet['alpha'] = files[file]['alpha']
        planet['k'] = files[file]['k']
        
    while t <= tmax:

        if method == 'dynamic': alpha, cp, k = get.Tdep_thermals(thermals, Tp)

        viscT = get.viscosity(planet, Tp)
        Ra = get.rayleigh(planet, Tp, Ts, viscT)

        production = evolve.produce_heat(planet, t)
        loss = evolve.flux_heat(planet, Tp, Ra)
        dTp = (dt * get.seconds * (production - loss)) / (planet['Cp'] * planet['pm'] * planet['Vm'])

        # This is what columnkey applies to
        Hts.append([t, Tp, Ra, production, loss, production / loss, planet['alpha'], planet['Cp'], Water, Mg, Si, Ca, Al, MgSi])

        Tp = Tp + dTp
        t = t + dt

    nplanets = nplanets + 1
    
    if nplanets%5==0:
        percentcomplete=Pf(100*nplanets/len(files.keys()))[:-5]+'%...'
        print(percentcomplete, end=" ")

    line = (file, Pf(MgSi), Pf(CaSi), Pf(AlSi), Pe(planet['alpha']), Pf(planet['Cp']), Pf(planet['k']), Pf(Tp), Pe(Ra), Pe(loss), Pf(production/loss), Pe(Water))
    out.write(sep.join(line))
    out.write('\n')

    line2 = (Pf(Mg), Pf(Si), Pf(Ca), Pf(Al), Pf(Tp), Pe(Water))
    out2.write(sep.join(line2))
    out2.write('\n')

print()
Evolution = pd.DataFrame(np.asarray(Hts), columns=columnkeys)
print('Done! Output file names: ' + output_file + '\t' + output_file_2)


### We can summarize our findings pretty easily...

In [ ]:
outcomes = pd.read_csv('compare_compositions_results.csv', sep=',')
outcomes.describe()

### ...But plots can better show how chemistry seems to affect thermal pathways.

In [ ]:
a=plot.evolution_colorcoded(Evolution, colorcolumn=ofinterest0, colortype='continuous')
b=plot.evolution_colorcoded(Evolution, colorcolumn=ofinterest1, colortype='continuous')
c=plot.evolution_colorcoded(Evolution, colorcolumn=ofinterest2, colortype='continuous')

### Plots can also show how factors correlate over time.

In [ ]:
cortime = px.scatter_matrix(Evolution, 
    dimensions=['time', 'temp', 'rayleigh', 'loss'],
    color='MgSi',
    color_discrete_sequence=px.colors.qualitative.Vivid,
    title="Scatter matrix of planet data set",
    labels=columnkeys)
cortime.update_layout(width=1000, height=1000)
cortime.update_traces(diagonal_visible=False, showupperhalf=False, marker=dict(size=2))
cortime.show()

### ...Or across parameters of interest.

In [ ]:
outcomes = pd.read_csv('compare_compositions_results.csv', sep=',')
corparams = px.scatter_matrix(outcomes, 
    dimensions=['Mg/Si', 'temp(K)', 'alpha', 'Cp', 'k', 'Water'],
    color="temp(K)",
    color_discrete_sequence=px.colors.qualitative.Vivid,
    title="Scatter matrix of planet data set",
    labels=columnkeys)
corparams.update_layout(width=1000, height=1000)
corparams.update_traces(diagonal_visible=False, showupperhalf=False, marker=dict(size=8))
corparams.show()

In [ ]:
corparallel = px.parallel_coordinates(outcomes, color='Mg/Si', color_continuous_scale='Rainbow', color_continuous_midpoint=np.mean(outcomes['Mg/Si']),
                              dimensions=['alpha', 'Cp', 'k', 'Water', 'temp(K)']).update_layout(template='ggplot2+presentation')
corparallel


### Want to save one of these interactive figures?
Copy, paste, and customize this in a "code" cell below (i.e. with figurename changed to indicate which you want, e.g. cortime, corparams, corparallel, etc.):


``import plotly.io as pio
pio.write_html(figurename, file='myfigure.html', auto_open=False)``